In [ ]:
!pip install efficientnet
!pip install livelossplot

# Import necessary libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import PIL
from IPython.display import Image, display
from keras.applications.vgg16 import VGG16,preprocess_input
# Plotly for the interactive viewer (see last section)
import plotly.graph_objs as go
import plotly.graph_objects as go
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model,load_model
from keras.applications.vgg16 import VGG16,preprocess_input
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten,BatchNormalization,Activation, Input
from tensorflow.keras.layers import GlobalMaxPooling2D
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import gc
import skimage.io
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
import efficientnet.tfkeras as efn
from livelossplot import PlotLossesKeras

# Data Exploration

In [ ]:
train_dir = "/kaggle/input/siim-isic-melanoma-classification/jpeg/train/"
test_dir = "/kaggle/input/siim-isic-melanoma-classication/jpeg/train/"
train = pd.read_csv("/kaggle/input/siim-isic-melanoma-classification/train.csv")
test = pd.read_csv("/kaggle/input/siim-isic-melanoma-classification/test.csv")
submission = pd.read_csv("/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv")

In [ ]:
labels = train["benign_malignant"].value_counts().index
values = train["benign_malignant"].value_counts().values

trace = go.Pie(labels=labels, values=values)
go.Figure(data=[trace])

Dataset is unbalanced

In [ ]:
trace_benign = go.Box(y=train.query("benign_malignant=='benign'")["age_approx"], name="benign")
trace_malignant = go.Box(y=train.query("benign_malignant=='malignant'")["age_approx"], name="malignant")
data = [trace_benign, trace_malignant]
layout = go.Layout(title="age vs benign", xaxis_title="benign_malignant", yaxis_title="age")
go.Figure(data, layout)

Cannot see very much difference for age group

In [ ]:
labels = train["anatom_site_general_challenge"].value_counts().index
values = train["anatom_site_general_challenge"].value_counts().values
trace = go.Pie(labels=labels, values=values, textinfo="label+percent", insidetextorientation="radial")
fig = go.Figure(data=[trace])
fig.show()

In [ ]:
cb = pd.crosstab(train["anatom_site_general_challenge"], train["benign_malignant"])
cb["ratio"] = cb['benign'] / cb['malignant']
cb

Different anatom_site_general_challenge has different probability lead to the malignant

In [ ]:
labels = train["diagnosis"].value_counts().index
values = train["diagnosis"].value_counts().values
trace = go.Pie(labels=labels, values=values, textinfo="label+percent", insidetextorientation="radial")
fig = go.Figure(data=[trace])
fig.show()

In [ ]:
cb = pd.crosstab(train["diagnosis"], train["benign_malignant"])
cb["ratio"] = cb["benign"] / cb["malignant"]
cb

No wrong diagnoise

In [ ]:
df_0 = train[train['target']==0].sample(2000)
df_1 = train[train['target']==1]
train_downsample = pd.concat([df_0, df_1]).reset_index()

In [ ]:
malignant = []
sample = df_1.sample(40)
for i in sample.image_name:
    img = cv2.imread(train_dir + i + '.jpg')
    img = cv2.resize(img, (384, 384))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype('float32') / 255.
    malignant.append(img)
fx, ax = plt.subplots(5, 8, figsize=(10,8))
for i, img in enumerate(malignant):
    ax[i//8, i%8].imshow(img)
    ax[i//8, i%8].axis("off")
plt.show()

In [ ]:
labels = []
data = []
for i in range(train.shape[0]):
    data.append(train_dir + train["image_name"].iloc[i] + '.jpg')
    labels.append(train["target"].iloc[i])
df = pd.DataFrame(data)
df.columns = ["images"]
df["target"] = labels

In [ ]:
test_data=[]
for i in range(test.shape[0]):
    test_data.append(test_dir + test['image_name'].iloc[i]+'.jpg')
df_test=pd.DataFrame(test_data)
df_test.columns=['images']

# Process the input data

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df['images'],df['target'], test_size=0.2, random_state=1234)
train=pd.DataFrame(X_train)
train.columns=['images']
train['target']=y_train

validation=pd.DataFrame(X_val)
validation.columns=['images']
validation['target']=y_val

In [ ]:
nb_epochs = 40
batch_size  = 16*8
nb_train_steps = train.shape[0]//batch_size
nb_val_steps=validation.shape[0]//batch_size
print("Number of training and validation steps: {} and {}".format(nb_train_steps,nb_val_steps))

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,horizontal_flip=True)
val_datagen=ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
    train,
    x_col='images',
    y_col='target',
    target_size=(384, 384),
    batch_size=8,
    shuffle=True,
    class_mode='raw')

validation_generator = val_datagen.flow_from_dataframe(
    validation,
    x_col='images',
    y_col='target',
    target_size=(384, 384),
    shuffle=False,
    batch_size=8,
    class_mode='raw')

# Create Effcientnet with resolution (384, 384)

In [ ]:
def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed



In [ ]:
inp = tf.keras.layers.Input(shape = (384, 384, 3))
efnetb3 = efn.EfficientNetB3(weights = 'imagenet', include_top = False)
x = efnetb3(inp)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model = tf.keras.models.Model(inputs = [inp], outputs = [output])
opt = tf.keras.optimizers.Adam(learning_rate = 0.0003)
model.compile(
            optimizer = opt,
            loss = [binary_focal_loss(gamma = 2.0, alpha = 0.80)],
            metrics = [tf.keras.metrics.AUC()]
        )

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_auc', mode = 'max', patience = 5, 
                                                      verbose = 1, min_delta = 0.0001, restore_best_weights = True)

cb_lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_auc', factor = 0.4, patience = 2, verbose = 1, min_delta = 0.0001, mode = 'max') 
model.fit_generator(train_generator,
                   epochs=40,
                   validation_data=validation_generator)

In [ ]:
target=[]
for path in df_test['images']:
    img=cv2.imread(str(path))
    img = cv2.resize(img, (384,384))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    img=np.reshape(img,(1,384,384,3))
    prediction=model.predict(img)
    target.append(prediction[0][0])

submission['target']=target
    

# Submission

In [ ]:
submission.to_csv('sub_EfficientNetB3_384.csv', index = False)